In [52]:
import pandas as pd
import numpy as np

#### Задание 1
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [26]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [71]:
# группируем по userId и сразу применяем 3 функции

A = ratings.groupby('userId').agg(['count','min', 'max'])['timestamp'].reset_index()

A.head()

,userId,count,min,max
0,1,20,1260759108,1260759205
1,2,76,835355395,835356246
2,3,51,1298861589,1298932787
3,4,204,949778714,949982274
4,5,100,1163373044,1163375145


In [81]:
# вычисляем время жизни пользователя в новом столбце diff

A['diff'] = A['max'] - A['min']

A.head()

,userId,count,min,max,diff
0,1,20,1260759108,1260759205,97
1,2,76,835355395,835356246,851
2,3,51,1298861589,1298932787,71198
3,4,204,949778714,949982274,203560
4,5,100,1163373044,1163375145,2101


In [82]:
# 1 способ
# выборка тех, у кого более 100 оценок, считаем среднее и переводим секунды в дни

A[ (A['count'] > 100) ].mean()/3600/24

userId        0.003990
count         0.003611
min       13018.003185
max       13481.897947
diff        463.894762
dtype: float64

In [83]:
# 2 способ

A.loc[A['count'] > 100]['diff'].mean()/3600/24

463.89476214829165

#### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

аналогичную таблицу по типам выручки с указанием адреса клиента.

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [10]:
Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [46]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [47]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [48]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [59]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33


In [61]:
# таблица с тремя типами выручки для каждого client_id без указания адреса, 


dat = (rzd
       .merge(auto, on = 'client_id', how = 'outer')
       .merge(air, on = 'client_id', how = 'outer')
      )

# если нужно для дальнейших расчетов, заменяем NaN на 0, 
# если нет статистики, то скорее всего перевозки не было

dat.loc[pd.isnull(dat.rzd_revenue), 'rzd_revenue'] = 0
dat.loc[pd.isnull(dat.auto_revenue), 'auto_revenue'] = 0
dat.loc[pd.isnull(dat.air_revenue), 'air_revenue'] = 0

dat

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [63]:
dat_adress = client_base.merge(dat, on = 'client_id', how = 'outer')

# аналогично, если требуется, можно заменить нулевыми значениями данные перевозок NaN

# при таком типе объединения (merge -> outer) все данные по превозкам сохранаются, даже если 
#  в базе клиентов нет адреса клиента (специально удалила одного клиента из базы клиентов). 
# Как мне кажется это логично, т.к. адрес всегда можно дополнить.

dat_adress.loc[pd.isnull(dat_adress.address), 'address'] = 'заполнить адрес'

dat_adress

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,заполнить адрес,0.0,0.0,173.0


#### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. 
Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [ ]:
"""
Если я правильно поняла задачу, нужно поразмышлять о применении координат в аналитике.

Информация по координатам пользователей позволит сделать региональный анализ визитов и покупок 
и выделить наиболее и наименее активные регионы по визитам / покупкам.

Также расположение пользователя, который сделал покупку позволит системе логистики определить, 
с какого склада отправлять товар ближе (если покупка на сайте произошло, но товар еще не отправлен).

Также удаленность координат одного и того же пользователя может подсказать, что пользователь 
путешественник и для него можно посотрить рекомендательную систему или тематическую рекламу 
именно тех товаров, которые актуальны для людей в разъездах.

Можно, например, сделать словарь с широтами и долготами, по которым определить, к какому региону 
координаты отнести.

Но остается вопрос, какие координаты из набора координат взять (если их несколько).

Если еще имеются временные метки для визита/покупки и для координат, то кординаты можно 
сопоставить визиту и покупке по совпадающему времени.

Если все координаты пользователя из одного региона, то нам подойдет любая пара координат 
(если регионы дальше не дробим). Тогда берем, например, первую пару в наборе.

Если координаты в разных регионах, то можно вычислить равноудаленную от них точку и взять 
ее координаты за основу.

Для выборки путешественников (кто перемещается на большие расстояния) можно искать самое больше расстание
между парами координат в наборе координат конкретного пользователя.

"""